In [10]:
import pandas as pd
import numpy as np
import ast
import re
from zhon.hanzi import non_stops, stops
import os
import pickle
from opencc import OpenCC
# from transformers import RobertaForTokenClassification
import torch
import torch.nn.functional as F
import pandas as pd
from transformers import RobertaTokenizer
from transformers import BertTokenizer , BertConfig , BertModel , XLNetTokenizer, XLNetConfig , XLNetModel

import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import BertForSequenceClassification,BertForPreTraining
from torch.autograd import Variable

from torch import nn

cc = OpenCC('t2s')


In [11]:
# train_df = pd.read_csv("./tbrain/tbrain_train_split.csv")
train_df = pd.read_csv("./dataset/tbrain_train (3).csv")
train_df = train_df.fillna('None')

# test_df = pd.read_csv("./tbrain/tbrain_test_split.csv")
test_df = pd.read_csv("./dataset/tbrain_test (3).csv")
test_df = test_df.fillna('None')


# train_df = pd.concat([train_df,test_df])

train_df = train_df[train_df['name']!='[]']


# train_df = pd.read_csv('./tbrain/only_have_names_data.csv')
# train_df = train_df.fillna('None')
# train_df = train_df.sample(frac=1)

print(train_df.shape)
print(test_df.shape)


(335, 4)
(491, 4)


In [12]:
def clean_string(content):
#     cc = OpenCC('t2s')
    content = content.replace('    ','')
    content = content.replace('\n','。').replace('\t','，').replace('!', '！').replace('?', '？')# erease white space cause English name error
    content = re.sub("[+\.\/_,$%●▼►^*(+\"\']+|[+——~@#￥%……&*（）★]", "",content)
    content = re.sub(r"[%s]+" %stops, "。",content)
    
#     content = cc.convert(content)
    return content

def cut_sent(para):
    para = re.sub('([。！？\?])([^”’])', r"\1\n\2", para)
    para = re.sub('(\.{6})([^”’])', r"\1\n\2", para) 
    para = re.sub('(\…{2})([^”’])', r"\1\n\2", para)  
    para = re.sub('([。！？\?][”’])([^，。！？\?])', r'\1\n\2', para)
    return para.split("\n")


def combine_sentence(sentences):
    li = []
    string = ''
    for k in range(len(sentences)):
        sentence = sentences[k]
        if len(string) + len(sentence) < 510:
            string = string + sentence
        else:
#             原本是空的代表sentences太常
            if string == '':
                n = 510
                tmp_li = [sentence[i:i+n] for i in range(0, len(sentence), n)]
                string = tmp_li.pop(-1)
                li = li + tmp_li
            else:
                li.append(string)
                string = sentence
    if(string != ''):
        li.append(string)
    return li

            
    

In [4]:
#### 切塊且找全部人名的位置  42k


names =  train_df['name'].tolist()
contents = np.array(train_df['full_content'].tolist())
ckip_names = train_df['ckip_names'].tolist()
x = []
binary_y = []
names_y  = []
content_all_names = []
start_pos_labels = []
end_pos_labels = []
for i in range(len(contents)):
    content = contents[i]
    content = clean_string(content)
    content_ckip_names = ast.literal_eval(ckip_names[i])

    if(content=='nan'):
        continue

    name_li  = ast.literal_eval(names[i])
    if(len(name_li) == 0 ):
        continue
#         content切句 
        split_content = cut_sent(content)
        chunks = combine_sentence(split_content)
        
        for chunk in chunks:
            start_pos_label = np.zeros(512)
            end_pos_label = np.zeros(512)
            binary_y.append(0)
            start_pos_label[0] = 1
            end_pos_label[0] = 1
            x.append(chunk)
            start_pos_labels.append(start_pos_label)
            end_pos_labels.append(end_pos_label)
            content_all_names.append(content_ckip_names)
        
    else:
        
        for name in name_li:
            name = name.replace('\n','').replace('\t','').replace(' ','')
            #  content切句 
            split_content = cut_sent(content)
            chunks = combine_sentence(split_content)

            for chunk in chunks:
                _pos = 0
                while True:
                    start_pos = chunk.find(name,_pos)
                    if(start_pos == -1):
                        break
                    start_pos += 1
                    _pos = start_pos
                    end_pos = start_pos + len(name)

                    if(start_pos < 512 and end_pos < 512 ):
                        binary_y.append(1)
                        x.append(chunk)

                        start_pos_label = np.zeros(512)
                        end_pos_label = np.zeros(512)
                        start_pos_label[start_pos] = 1
                        end_pos_label[end_pos] = 1

                        start_pos_labels.append(start_pos_label)
                        end_pos_labels.append(end_pos_label)
                        content_all_names.append(content_ckip_names)
        
        
train_x = np.array(x)
train_binary_y = np.array(binary_y)
train_start_pos_labels = np.array(start_pos_labels)
train_end_pos_labels = np.array(end_pos_labels)
train_ckip_names = np.array(content_all_names)


print(train_x.shape)
print(train_binary_y.shape)
print(train_start_pos_labels.shape)
print(train_end_pos_labels.shape)
print(train_ckip_names.shape)


(2857,)
(2857,)
(2857, 512)
(2857, 512)
(2857,)


In [9]:
with open('./TB/train_x.pickle', 'wb') as handle:
    pickle.dump(train_x, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('./TB/train_binary_y.pickle', 'wb') as handle:
    pickle.dump(train_binary_y, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('./TB/train_start_pos_labels.pickle', 'wb') as handle:
    pickle.dump(train_start_pos_labels, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('./TB/train_end_pos_labels.pickle', 'wb') as handle:
    pickle.dump(train_end_pos_labels, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [20]:
with open('./TB/train_x.pickle', 'rb') as handle:
    train_x = pickle.load(handle)
    
with open('./TB/train_binary_y.pickle', 'rb') as handle:
    train_binary_y = pickle.load(handle)
    
with open('./TB/train_start_pos_labels.pickle', 'rb') as handle:
    train_start_pos_labels = pickle.load(handle)
    
with open('./TB/train_end_pos_labels.pickle', 'rb') as handle:
    train_end_pos_labels = pickle.load(handle)
print(train_x.shape)
print(train_binary_y.shape)
print(train_start_pos_labels.shape)
print(train_end_pos_labels.shape)

(7327,)
(7327,)
(7327, 512)
(7327, 512)


In [4]:
# model_path  = './chinese_xlnet_mid_pytorch/'
# model_path = './pretrain_roberta_on_TBdata/'

bert_pretrain = './bert_wwm_pretrain_tbrain/'
tokenizer = BertTokenizer.from_pretrained(bert_pretrain)
config = BertConfig.from_pretrained(bert_pretrain + 'config.json',output_hidden_states=True)



# model_path = './chinese_xlnet_mid_pytorch/'
# tokenizer = XLNetTokenizer.from_pretrained(model_path)


train_input_dict = tokenizer.batch_encode_plus(train_x, 
                                         add_special_tokens=True,
                                         max_length=512,
                                        truncation=True, 
                                         return_special_tokens_mask=True,
                                         pad_to_max_length=True,
                                         return_tensors='pt')


# with open('./TB/train_input_dict.pickle', 'wb') as handle:
#     pickle.dump(train_input_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)


NameError: name 'train_x' is not defined

In [6]:
# names =  test_df['name'].tolist()
# contents = np.array(test_df['full_content'].tolist())
# ckip_names = test_df['ckip_names'].tolist()

# x = []
# binary_y = []
# names_y  = []
# content_all_names = []

# start_pos_labels = []
# end_pos_labels = []
# for i in range(len(contents)):
#     content = contents[i]
#     content = clean_string(content)
#     content_ckip_names = ast.literal_eval(ckip_names[i])


#     if(content=='nan'):
#         continue
# #     name = names[i]
#     name_li  = ast.literal_eval(names[i])


    
#     if(len(name_li) == 0):
        
# #         content切句 
#         split_content = cut_sent(content)
#         chunks = combine_sentence(split_content)
        
#         for chunk in chunks:
#             start_pos_label = np.zeros(512)
#             end_pos_label = np.zeros(512)
#             binary_y.append(0)
#             start_pos_label[0] = 1
#             end_pos_label[0] = 1
#             x.append(chunk)
#             start_pos_labels.append(start_pos_label)
#             end_pos_labels.append(end_pos_label)
#             content_all_names.append(content_ckip_names)
        
#     else:
        
#         for name in name_li:
        
#             name = name.replace('\n','').replace('\t','').replace(' ','')

#             #  content切句 
#             split_content = cut_sent(content)
#             chunks = combine_sentence(split_content)

#             for chunk in chunks:


#                 _pos = 0
#                 while True:
#                     start_pos = chunk.find(name,_pos)
#                     if(start_pos == -1):
# #                         continue
#                         break
#                     start_pos += 1
#                     _pos = start_pos
#                     end_pos = start_pos + len(name)

#                     if(start_pos < 512 and end_pos < 512 ):
#                         binary_y.append(1)
#                         x.append(chunk)

#                         start_pos_label = np.zeros(512)
#                         end_pos_label = np.zeros(512)
#                         start_pos_label[start_pos] = 1
#                         end_pos_label[end_pos] = 1

#                         start_pos_labels.append(start_pos_label)
#                         end_pos_labels.append(end_pos_label)
#                         content_all_names.append(content_ckip_names)
        

    
# test_x = np.array(x)
# test_binary_y = np.array(binary_y)
# test_start_pos_labels = np.array(start_pos_labels)
# test_end_pos_labels = np.array(end_pos_labels)
# test_ckip_names = np.array(content_all_names)


# print(test_x.shape)
# print(test_binary_y.shape)
# print(test_start_pos_labels.shape)
# print(test_end_pos_labels.shape)
# print(test_ckip_names.shape)

In [27]:
with open('./TB/test_x.pickle', 'wb') as handle:
    pickle.dump(test_x, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('./TB/test_binary_y.pickle', 'wb') as handle:
    pickle.dump(test_binary_y, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('./TB/test_start_pos_labels.pickle', 'wb') as handle:
    pickle.dump(test_start_pos_labels, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('./TB/test_end_pos_labels.pickle', 'wb') as handle:
    pickle.dump(test_end_pos_labels, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [28]:
with open('./TB/test_x.pickle', 'rb') as handle:
    test_x = pickle.load(handle)
    
with open('./TB/test_binary_y.pickle', 'rb') as handle:
    test_binary_y = pickle.load(handle)
    
with open('./TB/test_start_pos_labels.pickle', 'rb') as handle:
    test_start_pos_labels = pickle.load(handle)
    
with open('./TB/test_end_pos_labels.pickle', 'rb') as handle:
    test_end_pos_labels = pickle.load(handle)
print(test_x.shape)
print(test_binary_y.shape)
print(test_start_pos_labels.shape)
print(test_end_pos_labels.shape)

(773,)
(773,)
(773, 512)
(773, 512)


In [20]:
test_input_dict = tokenizer.batch_encode_plus(test_x, 
                                         add_special_tokens=True,
                                         max_length=512,
                                         return_special_tokens_mask=True,
                                         pad_to_max_length=True,
                                         return_tensors='pt')


# with open('./TB/test_input_dict.pickle', 'wb') as handle:
#     pickle.dump(test_input_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [5]:
class TrainDataset(Dataset):
    def __init__(self, input_dict, y , start_pos_labels_train, end_pos_labels_train):
        self.input_ids = input_dict['input_ids']
        self.token_type_ids = input_dict['token_type_ids']
        self.attention_mask = input_dict['attention_mask']
        self.y = y
        self.start_pos_labels = start_pos_labels_train
        self.end_pos_labels = end_pos_labels_train
    def __getitem__(self,idx):
        inputid = self.input_ids[idx]
        tokentype = self.token_type_ids[idx]
        attentionmask = self.attention_mask[idx]
        start_pos = self.start_pos_labels[idx]
        end_pos =  self.end_pos_labels[idx]
        y = self.y[idx]
        return inputid , tokentype , attentionmask, y , start_pos , end_pos
    
    def __len__(self):
        return len(self.input_ids)
    

In [6]:
class posClassfication_new(nn.Module):
    def __init__(self):
        super(posClassfication_new, self).__init__()
        self.start_task = nn.Sequential(
            nn.Linear(768, 1),
        )    
        self.end_task = nn.Sequential(
            nn.Linear(768, 1),
        ) 
        

#             
    def forward(self, start_x, end_x):
        start_x = start_x.double()
        end_x = end_x.double()
        
    
        start_out = self.start_task(start_x)
        end_out = self.end_task(end_x)
        
        return start_out , end_out 

In [11]:
BATCH_SIZE = 8
trainset = TrainDataset(train_input_dict, 
                        train_binary_y,
                        train_start_pos_labels,
                        train_end_pos_labels)

trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle = True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)


# model = XLNetModel.from_pretrained(model_path)

model = BertModel.from_pretrained(bert_pretrain,config=config)
model = model.to(device)
model.output_hidden_states = True
model.eval()

pos_model = posClassfication_new()
pos_model = pos_model.to(device)
pos_model = pos_model.double()
pos_model.train()
pos_loss = nn.CrossEntropyLoss()




EPOCHS = 10
optimizer = torch.optim.Adam(pos_model.parameters(), lr=3e-5)

for epoch in range(EPOCHS):
    running_loss = 0.0
    for data in trainloader:
        
        tokens_tensors ,  segments_tensors , masks_tensors , binary_y ,start_pos_labels , end_pos_labels = [t.to(device) for t in data]
        
        bert_outputs = model(input_ids=tokens_tensors, 
                                token_type_ids=segments_tensors, 
                                attention_mask=masks_tensors)
        
  

        bert_all_768 = bert_outputs[0]
        mini_batch = bert_all_768.size()[0]
        
        bert_all_768 = bert_all_768.double()
        optimizer.zero_grad()      
        start_pred, end_pred  = pos_model(bert_all_768, bert_all_768)
        
 
        start_pred = start_pred.reshape((mini_batch,512))
        end_pred = end_pred.reshape((mini_batch,512))
        
        
        start_pos_labels = torch.argmax(start_pos_labels,dim=1)
        end_pos_labels = torch.argmax(end_pos_labels,dim=1)
        
        start_loss = pos_loss(start_pred,start_pos_labels)
        end_loss = pos_loss(end_pred,end_pos_labels)
        
        loss = ((start_loss + end_loss) / 2)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
    torch.save(pos_model.state_dict(),'./TB_multispan/Bert_single_span_dataset3_' + str(epoch) + '.pkl')
    print('[epoch %d] loss: %.3f' %(epoch + 1, running_loss))



device: cuda:0
[epoch 1] loss: 2245.514
[epoch 2] loss: 2189.786
[epoch 3] loss: 2152.614
[epoch 4] loss: 2127.890
[epoch 5] loss: 2111.212
[epoch 6] loss: 2098.730
[epoch 7] loss: 2088.772
[epoch 8] loss: 2080.648
[epoch 9] loss: 2073.386
[epoch 10] loss: 2067.363


In [7]:
def eval(pred, ans):
    if bool(pred) is not bool(ans):
        return 0
    elif not pred and not ans:
        return 1
    else:
        pred = set(pred)
        ans = set(ans)
        interaction_len = len(pred & ans)
        if interaction_len == 0:
            return 0

        pred_len = len(pred)
        ans_len = len(ans)
        return 2 / (pred_len / interaction_len + ans_len / interaction_len)


def eval_all(pred_list, ans_list):
    assert len(pred_list) == len(ans_list)
    return sum(eval(p, a) for p, a in zip(pred_list, ans_list)) / len(pred_list)



In [8]:
def check_pred_name_is_real_ans(pred_name_list,news,dataset):
    class Testset(Dataset):
        def __init__(self, input_ids , token_type_ids , attention_mask):
            self.input_ids = input_ids
            self.token_type_ids = token_type_ids
            self.attention_mask = attention_mask
        def __getitem__(self,idx):
            inputid = self.input_ids[idx]
            tokentype = self.token_type_ids[idx]
            attentionmask = self.attention_mask[idx]

            return inputid , tokentype , attentionmask

        def __len__(self):
            return len(self.input_ids)
    
    lm_path = './bert_wwm_pretrain_tbrain/'
    tokenizer = BertTokenizer.from_pretrained(lm_path)

    content = clean_string(news)
    train_input_ids = []
    train_token_types = []
    train_attention_mask = []
        
    for name in pred_name_list:
        
        content_max_length = 512-3-len(name)
        
        if len(content) >= content_max_length:
            content = content[:content_max_length]
            
        input_ids = tokenizer.encode(name, content)
        if(len(input_ids)>512):
            continue
        sep_index = input_ids.index(tokenizer.sep_token_id)
        num_seg_a = sep_index + 1
        num_seg_b = len(input_ids) - num_seg_a
        segment_ids = [0]*num_seg_a + [1]*num_seg_b

        input_mask = [1] * len(input_ids)

        while len(input_ids) < 512:
            input_ids.append(0)
            input_mask.append(0)
            segment_ids.append(0)
            
        train_input_ids.append(input_ids)
        train_token_types.append(segment_ids)
        train_attention_mask.append(input_mask)
        
    train_input_ids = np.array(train_input_ids)
    train_token_types  = np.array(train_token_types)
    train_attention_mask = np.array(train_attention_mask)
    
    
    BATCH_SIZE = train_input_ids.shape[0]
    
    testset = Testset(train_input_ids ,train_token_types , train_attention_mask)
    testloader = DataLoader(testset, batch_size=BATCH_SIZE)
    
    
    from transformers import BertForSequenceClassification
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#     print("device:", device)

    lm_path = './bert_wwm_pretrain_tbrain/'
    NUM_LABELS = 2
    tokenizer = BertTokenizer.from_pretrained(lm_path)
    model = BertForSequenceClassification.from_pretrained(lm_path,num_labels=NUM_LABELS)
    
    check_point = ''
    if(dataset == 0):
#         最一開始的dataset
        check_point = './TB_multispan/Bert_wwm_ckip_name_is_ans_13.pkl'

    elif (dataset == 1):
#         tbrain_train (1).csv
        check_point = './TB_multispan/Bert_wwm_ckip_name_is_ans_dataset1_epoch17.pkl'

    elif (dataset == 2):
#         tbrain_train (2).csv
        check_point = './TB_multispan/Bert_wwm_ckip_name_is_ans_dataset2_epoch13.pkl'

    elif (dataset == 3):
#         tbrain_train (3).csv
        check_point = './TB_multispan/Bert_wwm_ckip_name_is_ans_dataset3_epoch18.pkl'

    elif (dataset == 4):
#        traindata + testdata
        check_point = './TB_multispan/Bert_wwm_ckip_name_is_ans_alldataset_epoch18.pkl'


    model.load_state_dict(torch.load(check_point))
    model = model.to(device)
    model.eval()


    with torch.no_grad():
        for data in testloader:
            tokens_tensors, segments_tensors, masks_tensors = [t.to(device) for t in data]
            outputs = model(input_ids=tokens_tensors, 
                                token_type_ids=segments_tensors, 
                                attention_mask=masks_tensors)
            pred = torch.softmax(outputs[0] , dim = -1)
            torch.set_printoptions(precision=10)
#             print(pred)
            pred = torch.argmax(pred,dim=-1)
            pred = pred.cpu().detach().numpy()
            pred_name_list = np.array(pred_name_list)
            return list(pred_name_list[pred>0])
        


In [9]:
def ckip_names_filter(name_list):
    lm_path = './pretrain_roberta_on_TBdata/'
    name_model_state_path = './TB_multispan/name_model_state.pkl'
    
    name_model = NameModel()
    name_model.load_state_dict(torch.load(name_model_state_path))
    name_model = name_model.double()
    name_model = name_model.to(device)
    name_model.eval()
    
    
    tokenizer = BertTokenizer.from_pretrained(lm_path)

    test_input_dict = tokenizer.batch_encode_plus(name_list, 
                                         add_special_tokens=True,
                                         max_length=5,
                                        truncation=True,
                                         return_special_tokens_mask=True,
                                         pad_to_max_length=True,
                                         return_tensors='pt')

    model = BertModel.from_pretrained(lm_path)
    model = model.to(device)
    model.output_hidden_states = True
    model.eval()
    

    bert_outputs = model(input_ids = test_input_dict['input_ids'].to(device), 
                        token_type_ids = test_input_dict['token_type_ids'].to(device), 
                        attention_mask= test_input_dict['attention_mask'].to(device))  

    pool_cls = bert_outputs[1]
    pool_cls = pool_cls


    logits = name_model(pool_cls)
    logits = torch.argmax(logits,dim = -1)
    logits = logits.cpu().detach().numpy()
    name_list = np.array(name_list)
    
    return list(name_list[logits>0])
    

class NameModel(nn.Module):
    def __init__(self):
        super(NameModel, self).__init__()
        self.name_task = nn.Sequential(
            nn.Linear(768, 768),
            nn.Tanh(),
            nn.Linear(768,2)
        )    
         
    def forward(self, x):
        x = x.double()
        out = self.name_task(x)
        return out



    

In [10]:
# testing
def two_words_is_name(name_list):
    from transformers import BertForSequenceClassification
    class TestDataset(Dataset):
        def __init__(self, input_dict ):
            self.input_ids = input_dict['input_ids']
            self.token_type_ids = input_dict['token_type_ids']
            self.attention_mask = input_dict['attention_mask']
        def __getitem__(self,idx):
            inputid = self.input_ids[idx]
            tokentype = self.token_type_ids[idx]
            attentionmask = self.attention_mask[idx]
            return inputid , tokentype , attentionmask
        def __len__(self):
            return len(self.input_ids)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


    lm_path = './bert_wwm_pretrain_tbrain/'
    tokenizer = BertTokenizer.from_pretrained(lm_path)


    input_dict = tokenizer.batch_encode_plus(name_list, 
                                         add_special_tokens=True,
                                         max_length=4,
                                       truncation=True,
                                         return_special_tokens_mask=True,
                                         pad_to_max_length=True,
                                         return_tensors='pt')
    BATCH_SIZE = len(name_list)
    testset = TestDataset(input_dict)
    testloader = DataLoader(testset, batch_size=BATCH_SIZE)

    NUM_LABELS = 2
    model = BertForSequenceClassification.from_pretrained(lm_path,num_labels=NUM_LABELS)
    check_point = './TB_multispan/Bert_wwm_name_model_2words_all_2.pkl'
    model.load_state_dict(torch.load(check_point))
    model = model.to(device)
    model.eval()


    with torch.no_grad():
        for data in testloader:
            tokens_tensors, segments_tensors, \
            masks_tensors = [t.to(device) for t in data]
            outputs = model(input_ids=tokens_tensors, 
                                token_type_ids=segments_tensors, 
                                attention_mask=masks_tensors)
            
            pred = torch.argmax(outputs[0],dim=-1)
            pred = pred.cpu().detach().numpy()
            name_list = np.array(name_list)
            return list(name_list[pred>0])


In [16]:
# calcuate test


BATCH_SIZE = 1
class TestDataset(Dataset):
    def __init__(self, input_dict,text):
        self.input_ids = input_dict['input_ids']
        self.token_type_ids = input_dict['token_type_ids']
        self.attention_mask = input_dict['attention_mask']
        self.text = text

    def __getitem__(self,idx):
        inputid = self.input_ids[idx]
        tokentype = self.token_type_ids[idx]
        attentionmask = self.attention_mask[idx]
        text = self.text[idx]

        return inputid , tokentype , attentionmask , text
    
    def __len__(self):
        return len(self.input_ids)
    
# class NameModel(nn.Module):
#     def __init__(self):
#         super(NameModel, self).__init__()
#         self.name_task = nn.Sequential(
#             nn.Linear(768, 768),
#             nn.Tanh(),
#             nn.Linear(768,2)
#         )    
         
#     def forward(self, x):
#         x = x.double()
#         out = self.name_task(x)
#         return out
    
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = 'cpu'
print("device:", device)



bert_pretrain = './bert_wwm_pretrain_tbrain/'
tokenizer = BertTokenizer.from_pretrained(bert_pretrain)
config = BertConfig.from_pretrained(bert_pretrain + 'config.json',output_hidden_states=True)
model = BertModel.from_pretrained(bert_pretrain,config=config)


# model_path = './chinese_xlnet_mid_pytorch/'
# tokenizer = XLNetTokenizer.from_pretrained(model_path)
# model = XLNetModel.from_pretrained(model_path)


model = model.to(device)
model.output_hidden_states = True
model.eval()


pos_model = posClassfication_new()
pos_model = pos_model.double()
pos_model.load_state_dict(torch.load('./TB_multispan/Bert_single_span_dataset1_0.pkl'))
pos_model = pos_model.to(device)
pos_model.eval()


# name_model = torch.load('./TB_multispan/name_model.pkl') 
# name_model = name_model.to(device)
# name_model.eval()

all_scroe = 0
names =  test_df['name'].tolist()
contents = np.array(test_df['full_content'].tolist())
ckip_names = test_df['ckip_names'].tolist()

x = []
binary_y = []
names_y  = []
content_all_names = []

start_pos_labels = []
end_pos_labels = []


valid_count = 0
score = 0.0

for i in range(len(contents)):
    content = contents[i]
    content = clean_string(content)
    content_ckip_names = ast.literal_eval(ckip_names[i])

    if(content=='nan'):
        continue
    name_li  = ast.literal_eval(names[i])
    my_pred_name_list = []
    two_word_pred_name_list = []
    
    if len(name_li) == 0:
        continue
        
    

    split_content = cut_sent(content)
    chunks = combine_sentence(split_content)

    test_input_dict = tokenizer.batch_encode_plus(chunks, 
                                     add_special_tokens=True,
                                     max_length=512,
                                    truncation=True,
                                     return_special_tokens_mask=True,
                                     pad_to_max_length=512,
                                     return_tensors='pt')

    testset = TestDataset(test_input_dict,chunks)
    testloader = DataLoader(testset, batch_size=BATCH_SIZE)


    for data in testloader:

        tokens_tensors ,  segments_tensors , masks_tensors = [t.to(device) for t in data[:-1]]
        text = data[-1]
        bert_outputs = model(input_ids=tokens_tensors, 
                            token_type_ids=segments_tensors, 
                            attention_mask=masks_tensors)
        

        bert_all_768 = bert_outputs[0]

        mini_batch = bert_all_768.size()[0]
        bert_all_768 = bert_all_768.double()

        start_pred , end_pred  = pos_model(bert_all_768, bert_all_768)

            
        start_pred = start_pred.reshape((mini_batch,512))
        end_pred = end_pred.reshape((mini_batch,512))
        
        
        
        start_pred = start_pred[0]
        end_pred = end_pred[0]
        
        start_pred = torch.softmax(start_pred,dim=0)
        end_pred = torch.softmax(end_pred,dim=0)
        


        topk = 5
        myrange = 20
        start_topk_indices = torch.topk(start_pred, topk).indices
        end_top_k_indices = torch.topk(end_pred,topk).indices
        all_indices = torch.cat([start_topk_indices,end_top_k_indices] , dim=-1)
        all_indices = torch.unique(all_indices)

        for i in range(all_indices.size()[0]):
            start_index = all_indices[i]
            start_index -= 1
            ans_string = ''

            if (start_index + myrange < 512) and (start_index - myrange > 0):
                ans_string = text[0][start_index - myrange:start_index + myrange]
                
            elif start_index + myrange >= 512:
                ans_string = text[0][start_index-myrange:-1]
                
            elif (start_index - myrange) <= 0 :
                ans_string = text[0][0:start_index+myrange]
            else:
                print('out of range')
                


            for ckip_name in content_ckip_names:
                if (ckip_name in ans_string and len(ckip_name) == 2):
                    two_word_pred_name_list.append(ckip_name)
                    
                elif(ckip_name in ans_string and len(ckip_name) > 2):
                    my_pred_name_list.append(ckip_name)

    print('----------------')
    print('before filter : ', set(my_pred_name_list))
    
    
    if(len(two_word_pred_name_list)>0):
        two_word_pred_name_list = two_words_is_name(two_word_pred_name_list)
    
    my_pred_name_list = my_pred_name_list + two_word_pred_name_list

    
    if len(my_pred_name_list)>0:
#         my_pred_name_list = ckip_names_filter(my_pred_name_list)
        my_pred_name_list = check_pred_name_is_real_ans(my_pred_name_list,content,1)
    
    
    print('pred : ' ,set(my_pred_name_list))
    print('ans : ' ,name_li)
    score += eval(my_pred_name_list,name_li)
    valid_count += 1
        

print(score/valid_count)
print(valid_count)
        


device: cuda:0
----------------
before filter :  {'賴俊吉'}


Some weights of the model checkpoint at ./bert_wwm_pretrain_tbrain/ were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were no

pred :  {'賴俊吉'}
ans :  ['賴俊吉']
----------------
before filter :  {'張銘坤', '陳揚宗'}


Some weights of the model checkpoint at ./bert_wwm_pretrain_tbrain/ were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were no

pred :  {'張銘坤', '陳揚宗'}
ans :  ['陳揚宗']
----------------
before filter :  {'王益洲'}


...

pred :  {'陳玟叡', '吳承霖'}
ans :  ['吳承霖', '陳玟叡']
----------------
before filter :  {'何壽川', '莊琇媛'}


Some weights of the model checkpoint at ./bert_wwm_pretrain_tbrain/ were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were no

pred :  {'何壽川'}
ans :  ['何壽川']
----------------
before filter :  {'陳清江', '蕭博文'}


Some weights of the model checkpoint at ./bert_wwm_pretrain_tbrain/ were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were no

pred :  {'陳清江'}
ans :  ['陳清江']
0.8114114114114115
37


In [16]:
BATCH_SIZE = 32
# trainset = TrainDataset(test_input_dict,test_binary_y,test_start_pos_labels,test_end_pos_labels)
trainset = TrainDataset(train_input_dict, 
                        train_binary_y,
                        train_start_pos_labels,
                        train_end_pos_labels)

trainloader = DataLoader(trainset, batch_size=BATCH_SIZE)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)

model = BertModel.from_pretrained('./pretrain_roberta_on_TBdata/',config=config)
model = model.to(device)
model.output_hidden_states = True
model.eval()

cls_x = np.array([])
all_768_x = np.array([])

flag = True
with torch.no_grad():
    for data in trainloader:
        
        tokens_tensors ,  segments_tensors , masks_tensors = [t.to(device) for t in data][:3]
        
        
        bert_outputs = model(input_ids=tokens_tensors, 
                                token_type_ids=segments_tensors, 
                                attention_mask=masks_tensors)
        
        
        


        hidden_state = bert_outputs[2]     
        
        bert_all_768 = bert_outputs[0]
        bert_all_768 = bert_all_768.double()
        tmp_768 = bert_all_768.clone().to('cpu').numpy()
        
        cls = bert_outputs[1]
        cls_tmp = cls.clone().to('cpu').numpy()

        if flag:
            flag = False
            cls_x = cls_tmp
            all_768_x = tmp_768
        else:
            cls_x = np.concatenate((cls_x,cls_tmp),axis = 0)
            all_768_x = np.concatenate((all_768_x,tmp_768),axis = 0)
            
# print(cls_x.shape)
# print(all_768_x.shape)


        
# # with open('./TB/train_all_768_x.pickle', 'wb') as handle:
# #     pickle.dump(all_768_x, handle, protocol=pickle.HIGHEST_PROTOCOL) 
# # with open('./TB/train_cls_x.pickle', 'wb') as handle:
# #     pickle.dump(cls_x, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
# with open('./TB/test_all_768_x.pickle', 'wb') as handle:
#     pickle.dump(all_768_x, handle, protocol=pickle.HIGHEST_PROTOCOL) 
# with open('./TB/test_cls_x.pickle', 'wb') as handle:
#     pickle.dump(cls_x, handle, protocol=pickle.HIGHEST_PROTOCOL)



device: cuda:0


KeyboardInterrupt: 

In [2]:
with open('./TB/train_all_768_x.pickle', 'rb') as handle:
    train_all_768_x = pickle.load(handle)
with open('./TB/train_cls_x.pickle', 'rb') as handle:
    train_cls_x = pickle.load(handle)
    
print(train_cls_x.shape)
print(train_all_768_x.shape)

(4890, 768)
(4890, 512, 768)


In [77]:
with open('./TB/test_all_768_x.pickle', 'rb') as handle:
    test_all_768_x = pickle.load(handle)
with open('./TB/test_cls_x.pickle', 'rb') as handle:
    test_cls_x = pickle.load(handle)
print(test_cls_x.shape)
print(test_all_768_x.shape)

(539, 768)
(539, 512, 768)


In [11]:
class binaryClassification(nn.Module):
    def __init__(self):
        super(binaryClassification, self).__init__()
        self.layer_1 = nn.Linear(768,2) 
#         self.layer_2 = nn.Linear(512, 256)
#         self.layer_3 = nn.Linear(256, 64) 
#         self.layer_4 = nn.Linear(64, 2) 
#         self.relu = nn.ReLU()
#         self.layer_1 = nn.Linear(768,2)
#             
    def forward(self, inputs):
#         inputs.requires_grad = True
        inputs = inputs.float()
#         x = self.relu(self.layer_1(inputs))
#         x = self.relu(self.layer_2(x))
#         x = self.relu(self.layer_3(x))
#         x = self.layer_4(x)
        x = self.layer_1(inputs)
        return x

In [12]:
class posClassfication_new(nn.Module):
    def __init__(self):
        super(posClassfication_new, self).__init__()
        self.start_task = nn.Sequential(
            nn.Linear(768, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )    
        self.end_task = nn.Sequential(
            nn.Linear(768, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        ) 
#             
    def forward(self, start_x,end_x):
        start_x = start_x.double()
        end_x = end_x.double()
        
        start_out = self.start_task(start_x)
        end_out = self.end_task(end_x)
        
        return start_out,end_out
        
        
        

In [13]:
# NUM_LABELS = 2
# tokenizer = BertTokenizer.from_pretrained('./chinese_roberta_wwm/')
# model = BertForSequenceClassification.from_pretrained('./chinese_roberta_wwm/',num_labels=NUM_LABELS)

model_name = './pretrain_roberta_on_TBdata'
tokenizer = BertTokenizer.from_pretrained(model_name)
config = BertConfig.from_pretrained(model_name,output_hidden_states=True)
# model = BertForSequenceClassification.from_pretrained('./chinese_roberta_wwm/pytorch_model.bin', 
#                                                       config=config)
# model.num_labels = 2


In [ ]:
# train_input_dict = tokenizer.batch_encode_plus(train_x, 
#                                          add_special_tokens=True,
#                                          max_length=512,
#                                          return_special_tokens_masks=True,
#                                          pad_to_max_length=True,
#                                          return_tensors='pt')

# with open('./TB/train_input_dict.pickle', 'wb') as handle:
#     pickle.dump(train_input_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
# # with open('./TB/train_input_dict.pickle', 'rb') as handle:
# #     train_input_dict = pickle.load(handle)

In [14]:
BATCH_SIZE = 8
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)

class BinaryClassifyTrainDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __getitem__(self,idx):
        x = self.x[idx]
        y = self.y[idx]
        return x, y
    
    def __len__(self):
        return len(self.x)
    
trainset = BinaryClassifyTrainDataset(train_cls_x , train_binary_y)
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE)

bi_model = binaryClassification()
bi_model = bi_model.to(device)
bi_model.train()

# weights = torch.tensor([0.15,1])
# weights = weights.to(device)
# weights = weights.float()


binary_loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(bi_model.parameters(), lr=1e-5)

EPOCHS = 20
for epoch in range(EPOCHS):
    step = 0
    running_loss = 0.0
    for data in trainloader:
        x, labels = [t.to(device) for t in data]
        optimizer.zero_grad()
        binary_classify_output = bi_model(x)
        labels = labels.long()
        loss = binary_loss(binary_classify_output,labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
    torch.save(bi_model,'binary_model_single_task_bertwwm.pkl')
    print('[epoch %d] loss: %.3f' %(epoch + 1, running_loss))

device: cuda:0


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type binaryClassification. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


[epoch 1] loss: 449.039
[epoch 2] loss: 383.626
[epoch 3] loss: 340.314
[epoch 4] loss: 307.652
[epoch 5] loss: 282.369
[epoch 6] loss: 262.339
[epoch 7] loss: 246.142
[epoch 8] loss: 232.805
[epoch 9] loss: 221.643
[epoch 10] loss: 212.159
[epoch 11] loss: 203.992
[epoch 12] loss: 196.870
[epoch 13] loss: 190.589
[epoch 14] loss: 184.995
[epoch 15] loss: 179.967
[epoch 16] loss: 175.412
[epoch 17] loss: 171.257
[epoch 18] loss: 167.441
[epoch 19] loss: 163.919
[epoch 20] loss: 160.651


In [15]:
bi_model.eval()

testset = BinaryClassifyTrainDataset(test_cls_x , test_binary_y)
testloader = DataLoader(testset, batch_size=BATCH_SIZE)
correct_count = 0
guess_1_count = 0
with torch.no_grad():
    for data in testloader:
        x, labels = [t.to(device) for t in data]
        binary_classify_output = bi_model(x)
        binary_classify_output = torch.softmax(binary_classify_output,dim = 1)
        binary_classify_output = torch.argmax(binary_classify_output,dim=1)
        correct_count += (labels == binary_classify_output).sum().item()
        guess_1_count += (binary_classify_output==1).sum().item()
        
print(correct_count, correct_count/test_cls_x.shape[0])
print(guess_1_count)

        

650 0.9193776520509194
234


In [80]:
BATCH_SIZE = 8
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)

class PosTrainDataset(Dataset):
    def __init__(self, x, start_y,end_y):
        self.x = x
        self.start_y = start_y
        self.end_y = end_y

    def __getitem__(self,idx):
        x = self.x[idx]
        start_y = self.start_y[idx]
        end_y = self.end_y[idx]
        return x, start_y, end_y
    
    def __len__(self):
        return len(self.x)
    
trainset = PosTrainDataset(train_all_768_x , train_start_pos_labels , train_end_pos_labels)
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE)

pos_model = posClassfication_new()
pos_model = pos_model.to(device)
pos_model = pos_model.double()
pos_model.train()

pos_loss = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(pos_model.parameters(), lr=1e-5)

EPOCHS = 20
for epoch in range(EPOCHS):
    running_loss = 0.0
    for data in trainloader:
        x, start_pos_labels, end_pos_labels = [t.to(device) for t in data]
        mini_batch = x.size()[0]
        
        
        optimizer.zero_grad()
        start_pred , end_pred = pos_model(x,x)
        
        start_pred = start_pred.reshape((mini_batch,512))
        end_pred = end_pred.reshape((mini_batch,512))


#         start_pred = nn.functional.log_softmax(start_pred,dim=1)
#         end_pred = nn.functional.log_softmax(end_pred,dim=1)
        
#         start_pos_labels = torch.softmax(start_pos_labels,dim=1)
#         end_pos_labels = torch.softmax(end_pos_labels,dim=1)
        
        start_pos_labels = torch.argmax(start_pos_labels,dim=1)
        end_pos_labels = torch.argmax(end_pos_labels,dim=1)

        start_loss = pos_loss(start_pred,start_pos_labels)
        end_loss = pos_loss(end_pred,end_pos_labels)
        
        
        loss = (start_loss + end_loss) / 2
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
    torch.save(pos_model,'pos_model_single_task.pkl')
    print('[epoch %d] loss: %.3f' %(epoch + 1, running_loss))

device: cuda:0
[epoch 1] loss: 2486.716
[epoch 2] loss: 1194.211
[epoch 3] loss: 1079.101
[epoch 4] loss: 1006.267
[epoch 5] loss: 960.555
[epoch 6] loss: 931.806
[epoch 7] loss: 912.169
[epoch 8] loss: 897.450
[epoch 9] loss: 885.691
[epoch 10] loss: 875.847
[epoch 11] loss: 867.385
[epoch 12] loss: 859.919
[epoch 13] loss: 853.207
[epoch 14] loss: 847.079
[epoch 15] loss: 841.424
[epoch 16] loss: 836.147
[epoch 17] loss: 831.169
[epoch 18] loss: 826.431
[epoch 19] loss: 821.887
[epoch 20] loss: 817.508


In [60]:
class PosTestDataset(Dataset):
    def __init__(self, x, start_y, end_y, binary_y, text):
        self.x = x
        self.start_y = start_y
        self.end_y = end_y
        self.text = text
        self.binary_y = binary_y

    def __getitem__(self,idx):
        x = self.x[idx]
        start_y = self.start_y[idx]
        end_y = self.end_y[idx]
        text = self.text[idx]
        binary_y = self.binary_y[idx]
        return x, start_y, end_y, binary_y , text
    
    def __len__(self):
        return len(self.x)

In [95]:
# method 1 512*512
BATCH_SIZE = 1
testset = PosTestDataset(test_all_768_x , test_start_pos_labels , test_end_pos_labels , test_binary_y, test_x)
testloader = DataLoader(testset, batch_size=BATCH_SIZE)
pos_model = pos_model.to(device)

pos_model.eval()



with torch.no_grad():
    for data in testloader:

        x, start_pos_labels, end_pos_labels, binary_y = [t.to(device) for t in data[:4]]
        text = data[4]
        mini_batch = x.size()[0]
        
    
        start_pred , end_pred = pos_model(x,x)
        
        start_pred = start_pred.reshape((mini_batch,512))
        end_pred = end_pred.reshape((mini_batch,512))
        
        
        start_pred = torch.nn.functional.softmax(start_pred,dim=1)
        end_pred = torch.nn.functional.softmax(end_pred,dim=1)
        
        
        
        
        
        start_pos_labels = torch.argmax(start_pos_labels,dim=1)
        end_pos_labels = torch.argmax(end_pos_labels,dim=1)
        

        
           
        if(binary_y[0] > 0):
            print('===========================')
            print(text[0])
            print('ans is: ', text[0][start_pos_labels-1:end_pos_labels-1])
            print('===========================')
        else:
            continue
        
#         start_topk_indices = torch.topk(start_pred, topk).indices
#         end_topk_indices = torch.topk(end_pred,topk).indices

        pred_mat = start_pred * end_pred.t()
        H, W = pred_mat.shape
        pred_mat = pred_mat.view(-1)
        topk = 3
        indices = torch.topk(pred_mat, topk).indices
        two_d_indices = torch.cat(((indices // W).unsqueeze(1), (indices % W).unsqueeze(1)), dim=1)

        for pos in two_d_indices:
            start_index = pos[1]-1
            end_index = pos[0]-1
            print(start_index,end_index)
            try:
                if(start_index < end_index):
                    print('model output ans ' + str(i) + ' ' +  text[0][start_index:end_index])
                elif(start_index == end_index):
                    print('model output ans ' + str(i) + ' ' +  text[0][start_index])
                else:
                    print('model output ans ' + str(i) + ' ' +  text[0][end_index:start_index])
                    
            except:
                print('out of range')
        
        

            




            
  
        
        
        
        

# print(start_correct , start_1)
# print(end_correct , end_1)


蘋果西打母公司掏空案總經理聲押禁見董座15萬交保大飲總經理孫幼英昨深夜移送新北檢複訊訊後遭聲押禁見記者陳慰慈攝2019-04-1206:00:12記者陳慰慈新北報導生產知名飲料蘋果西打的大西洋飲料公司1213因財報難產爆出掏空疑雲目前股票已暫停交易檢調昨依違反證券交易法非常規交易特別背信罪等罪約談大飲董事長江國貴登記負責人總經理孫幼英實際負責人獨立董事林偉強蘇芸樂財務經理鍾素娥張玉鳳等6人到案蘇芸樂昨晚訊後以3萬5千元交保今凌晨2時許依涉犯重罪有串證逃亡之虞將孫幼英聲押禁見江國貴15萬元林偉強10萬元鍾素娥40萬元張玉鳳25萬元交保大飲公司近日深陷財務經營風暴去年第3季與關係人簽約以64億元取得位於台南佳里高雄湖內及新北市新店等3筆不動產而與大飲交易的國信旭順公司董事名單高度重疊大飲總經理孫幼英身兼國信董事且國信又是大飲旭順的監察人旭順的監察人則是國信公司治理惹議請繼續往下閱讀此外國信旭順公司登記地址也是大飲位於新北市三重區的辦公室等於都是大飲能實質掌控的公司讓外界有左手賣右手的掏空質疑而林偉強蘇芸樂分別擔任國信食品代協理國信食品課長卻又是大飲獨立董事不符金管會規定讓會計師對大飲去年財報給出無法表達意見的結論導致財報必須重編股票也暫停交易檢調懷疑大飲透過土地交易搬錢解救虧損的國信旭順為釐清大飲遭質疑的3筆土地交易是以高買低或是買空賣空的假交易及金錢流向新北地檢署企業犯罪組主任檢察官鄧媛檢察官陳柏文昨指揮調查局新北市調處搜索大飲位於新北市三重區台北市中山區辦公室孫幼英住處等8地查扣相關帳冊等證物全案擴大偵辦中等證物全案擴大偵辦中不用抽不用搶現在用APP看新聞保證天天中獎
ans is:  林偉強
tensor(28, device='cuda:0') tensor(31, device='cuda:0')
model output ans 2 理孫幼
tensor(510, device='cuda:0') tensor(31, device='cuda:0')
model output ans 2 英昨深夜移送新北檢複訊訊後遭聲押禁見記者陳慰慈攝2019-04-1206:00:12記者陳慰慈新北報導生產知名飲料蘋果西打的大西洋飲料公司1213因財報難產爆出掏空疑雲目前股票已暫停交易檢調昨依違反證券交易法非常規交易特別背信罪等罪約談大飲董事長江國貴登記負責人總經理孫幼

國民黨前新竹市議員李黃錦燕更二審判刑4年資料照記者楊國文台北報導國民黨籍前新竹市議員李黃錦燕被控從2000年起利用人頭虛報議會補助的助理費近10年來合計領走932萬元一審依貪污治罪條例的利用職務詐取財務罪重判她10年且932萬元應予追繳發還市議會上訴後高等法院更一審曾以議會沒有審查公費助理工作內容的權限以罪證不足改判無罪不過高院更二審今上午又以李黃錦燕貪污罪行明確判她4年徒刑褫奪公權3年還可上訴李黃錦燕今上午未出庭聆判無法得知她對判決結果的看法法界指出因更一審曾判李黃錦燕無罪今更二審改判4年徒刑李黃錦燕上訴的可能性高請繼續往下閱讀李黃錦燕曾任新竹市第三到七屆議員任期到2010年2月8日為止後來競選連任失利檢警調查李黃錦燕從2000年起陸續向議會申報9人領取每月4萬元的助理費其中有些是李黃錦燕先生李榮勝診所的護士有些則是在外當保母縫紉工連她的兒子也名列虛報人頭之一而實際聘請的三名助理每月只領2萬元現金勞健保還須自付新竹地院審理時李黃錦燕指稱10年來所聘用的人都實際有擔任助理工作助理費都是用現金給他們有些助理認為錢給得太多希望她把一部分拿去做公益或是提供服務處開銷所有的錢都是助理同意奉獻的她付出的公益費用遠大於請領的費用不過法官認為李黃錦燕長期擔任民代竟以人頭助理或短報助理薪資方式詐領議會的公費助理薪資犯後飾詞狡辯矢口否認犯罪依貪污治罪條例中利用職務詐取財物罪重刑10年上訴後高等法院改判8年徒刑褫奪公權6年全案更一審時大逆轉改判李黃錦燕無罪檢方不服提起上訴經最高法院撤銷發回更二審今改判4年徒刑褫奪公權3年不用抽不用搶現在用APP看新聞保證天天中獎
ans is:  李黃錦燕
tensor(510, device='cuda:0') tensor(510, device='cuda:0')
model output ans 2 於
tensor(510, device='cuda:0') tensor(-1, device='cuda:0')
model output ans 2 
tensor(-1, device='cuda:0') tensor(510, device='cuda:0')
model output ans 2 
檢調偵辦潤寅涉嫌向國內14家銀行詐貸84億元案今再約談福懋越南廠經理王憲璋及新纖營業部部長李智剛到案訊後以2人涉嫌違反銀行法詐欺罪嫌重大有串

前職棒米迪亞暴龍隊老闆施允澤原名施建新遭控利用普萊普雷公司虛偽增資非常規交易挪用公司資產不實金流沖銷美化帳面數字從中獲利數千萬元新北地檢署16日依偽造文書背信等罪起訴施允澤及登記負責人吳茂達等人檢方調查原名施建新的施允澤為普萊普雷鑫奕國際等公司負責人施男涉嫌在2009年開始三度將普萊普雷公司虛偽增資讓外界誤認該公司具有資力另施允澤也遭控對外宣稱普萊普雷擁有關鍵字廣告專利邀請不知情人士投資2013年間施允澤又以普萊普雷和思柏公司合作開發燃料電池專利名義以1億5000萬元增資思柏公司並以5000萬元購買吳男手中的思柏股權但匯入思柏與吳男帳戶的錢隨即被轉出輾轉匯回施男帳戶但實際上這6項燃料電池相關專利並未經過合理鑑價亦無實際經濟收益普萊普雷公司也因此受到損害另外施男也遭控以對外投資交易以出售股權衛星設備方式輾轉挪用普萊普雷公司資產為彌補美化帳面數字施允澤挪用公司資金製作虛偽金流再利用層層轉匯手法沖銷挪用公司款項情事造成財報不實及股東損失另施允澤也被查出涉以不實的顧問費製造虛偽憑證新北地檢署歷經多年偵辦16日依偽造文書背信等罪嫌起訴施允澤吳茂達等7人中時)
ans is:  施建新
tensor(11, device='cuda:0') tensor(14, device='cuda:0')
model output ans 2 施允澤
tensor(11, device='cuda:0') tensor(19, device='cuda:0')
model output ans 2 施允澤原名施建新
tensor(16, device='cuda:0') tensor(14, device='cuda:0')
model output ans 2 原名
香港錦坤股權投資管理公司總裁李維凱被控以投資名義對外招攬民眾吸金吸金金額逾4億元涉嫌違反銀行法等罪嫌由於李男在案發前即逃匿香港日前遭檢方發布通緝法務部調查局已將李男列為追緝外逃要犯名單積極緝捕中調查局指出58歲的李維凱是台北市人自稱是香港錦坤股權投資管理有限公司總裁自2012年10月起在台北桃園台中台南高雄等地舉辦投資說明會公開向民眾招攬投資吸金吸金金額總計新台幣4億386萬餘元李維凱涉嫌違反銀行法證券交易法及刑法詐欺等罪嫌但李維凱早在2013年4月即出境未歸懷疑逃匿在大陸香港地區台中地檢署於今年7月31日依銀行法等罪嫌將

金山財神廟前主委鄭楠興侵占廟款2500萬餘元被訴金山財神廟前主任委員鄭楠興財務長黎美清利用執行廟務之便私自挪用廟款購地再將購得土地賣給廟方侵占金山財神廟款2503萬3551元基隆地檢署今天將2人依業務侵占罪嫌起訴記者林嘉東翻攝金山財神廟臉書2019-01-0214:55:17記者林嘉東基隆報導金山財神廟前主任委員鄭楠興財務長黎清美利用執行廟務之便私自挪用廟款購地再將購得土地賣給廟方侵占金山財神廟款2503萬3551元基隆地檢署今天將2人依業務侵占罪嫌起訴起訴書指出位於新北市萬里區磺潭里的金山財神廟原由呂姓男子興建呂89年間因積欠債務周轉不靈把原名為龍德廟的廟宇轉移給債權人經營龍德廟因此更名為金山財神廟請繼續往下閱讀金山財神廟前主任委員鄭楠興與財務長103年起兼任副主任委員的黎清美見金山財神廟信徒越來越多商機可期為擴大停車場空間與興建旅館供信徒投宿使用鄭黎於97年間未經財神廟委員會同意由鄭拿出500萬元訂金後簽立3200萬元本票以3700萬元價格向黃姓地主購得廟附近10筆土地檢調查出鄭黎為支付第二期與第三期土地價款共2000萬元98年間陸續侵占金山財神廟帳戶1700萬元還差300萬餘額由李姓監委出資鄭黎為籌措剩下1200萬元尾款透過劉姓女子向農會貸款1200萬元買下10筆土地100年間再侵占金山財神廟款803萬3551元2人合計共侵占金山財神廟2503萬3551元鄭黎將10筆土地合併成9筆土地後向金山財神廟謊稱其中7筆土地是他們2人出資購買願意賣給金山財神廟使廟方陷於錯誤花3126萬元向2人購得這7筆土地檢方認定鄭黎2侵占金山財神廟款的犯行觸犯業務侵占罪今天將2人依業務侵占罪嫌起訴不用抽不用搶現在用APP看新聞保證天天中獎
ans is:  黎清美
tensor(8, device='cuda:0') tensor(40, device='cuda:0')
model output ans 2 鄭楠興侵占廟款2500萬餘元被訴金山財神廟前主任委員鄭楠興財務長
tensor(8, device='cuda:0') tensor(39, device='cuda:0')
model output ans 2 鄭楠興侵占廟款2500萬餘元被訴金山財神廟前主任委員鄭楠興財務
tensor(8, device='cuda:0') tensor(19, device='cuda:

纏訟13年分時度假吸金13億英籍主謀更審判刑6年2006年時以分時度假為幌子的吸金詐騙集團吸金6年共詐得1億2942萬元經過纏訟13年高等法院更一審今依違反銀行法非法經營銀行業務罪對相關人等判刑資料照2019-05-3117:49:58記者張文川台北報導刑事警察局2006年破獲以分時度假為幌子的吸金詐騙集團吸金6年共詐得1億2942萬元經過纏訟13年高等法院更一審今依違反銀行法非法經營銀行業務罪加重改判英國籍主謀倪菲爾6年有期徒刑凌鳳琴等4名經理各判刑3年8月至4年半全案仍可上訴由於案件已審理逾8年適用速審法減刑規定因此倪菲爾等5人皆獲減刑但刑度仍比一審重台灣共犯劉穎谷陳志明各判刑4年6月凌鳳琴李泗源各判3年8月另一經理曾杭皆依常業詐欺罪判刑10月適用96年減刑條例減為5月6人犯罪不法所得一併宣告沒收請繼續往下閱讀判決指出倪菲爾等人於2001年2月起以樂吉美台灣分公司名義對外吸收CVC國際旅遊交換服務會員誘騙會員繳交2萬至70萬元會費宣稱會員可透過交換服務使用全球超過5500家渡假村並享受低廉的旅遊服務實則只是幌子集團還推出5年現金回饋計畫向會員宣稱5年後可以拿回本金或更高的金額實際上卻無法使會員領回會費使會員陷入錯誤而簽約付費集團2006年4月又推出新點子想再剝會員一層皮向會員推銷VIPASIA會員卡佯稱只須再繳交差額就可將先前繳納的CVC會費或其他公司的會籍款項連同這次的會費合併計算總金額後分割成每張價值4萬元的VIPASIA會員卡數張可對外轉售會員卡套利騙會員再次掏錢簽約高院認定樂吉美公司共吸金1億2942萬餘元被害人多達439多人卻未收購其他公司的分時度假權益CVC會員無法取得全球5500家渡假村的使用權也沒有所謂的優惠旅遊且樂吉美收取的會費價格紊亂毫無標準也無法解釋為何客戶先前付給其他公司的會費可以扣抵CVC會費且樂吉美的員工薪資人事費用龐大難以追查資金流向也沒有保證會員可5年取回費用的資金來源因此認定吉美樂違反銀行法詐騙吸金更一審認定倪菲爾等人同時觸犯銀行法與刑法常業詐欺罪從重依銀行法非法經營銀行業務罪論處判刑一審僅依常業詐欺罪輕判4年多未依銀行法論罪因此撤銷改判但因適用速審法給予減刑高院指出此案被害人多達439人不法吸金與詐騙逾億元對社會經濟秩序造成所重大危害公司負責人倪菲爾位居領導決策地位參與程度最深劉穎谷陳志明年薪1115萬陳志明年領逾400萬元2

記者黃捷台北報導毒販林輝宏1993年涉嫌以免費旅遊泰國為餌招攬6名男子吞食海洛因毒球方式從泰國走私毒品入台是國內首起吞毒球走私案後有2人因毒球在體內破裂毒發身亡林男事後遭控走私運毒及殺人罪但他逃亡已久運毒部分逾通緝時效台北地院判免訴殺人罪部分則因家屬聲請死亡宣告獲准台北地檢署認定林已死亡今處分不起訴檢方調查林輝宏與共犯魯德海1993年間找來張建華吳東明吳東明楊自立呂宗南裴振福劉吉雄以吞海洛因毒球走私為條件招待6人免費遊泰國若成功將毒品運回台灣可再另賺10萬元林輝宏先至泰國買毒並分裝成每顆5公克的毒球魯德海則協助代辦6男的出境手續6男抵達泰國後與林碰頭10月7日先讓呂宗南劉裴振福3人各吞下40多顆毒球後隔天搭機返台成功走私毒品同月11日林輝宏再將毒球交給張吳楊張吞20餘顆吳吞30餘顆楊則吞12顆但吳男吞下不久毒球就破裂當場毒發身亡張楊隔日搭機返台2人也因毒球破裂在飛機上身體不適2人抵台後在機場分手楊男自覺撐不住立刻請友人載他到醫院急診陸續排出11顆毒球撿回一命張男最後則中毒死亡檢警獲報後循線逮獲主嫌林輝宏魯德海及倖存的另4名運毒者北檢依懲治走私條例肅清煙毒條例起訴6人吳張則因死亡獲不起訴法院最後依販毒過失致死罪判魯無期徒刑4名運毒者則分別遭判4至10年不等徒刑林則因逃亡遭北院通緝林輝宏涉毒品罪部分今年4月已逾追訴時效北院判決免訴至於被告涉嫌殺人罪部分因林逃亡後至今音訊全無其家屬聲請死亡宣告獲准法律上認定林已死亡北檢今只能將他處分不起訴少一份毒品就多一份健康自由時報提醒您遠離毒品不用抽不用搶現在用APP看新聞保證天天中獎點我下載APP按我看活動辦法
ans is:  劉吉雄
tensor(183, device='cuda:0') tensor(183, device='cuda:0')
model output ans 2 呂
tensor(10, device='cuda:0') tensor(183, device='cuda:0')
model output ans 2 林輝宏1993年涉嫌以免費旅遊泰國為餌招攬6名男子吞食海洛因毒球方式從泰國走私毒品入台是國內首起吞毒球走私案後有2人因毒球在體內破裂毒發身亡林男事後遭控走私運毒及殺人罪但他逃亡已久運毒部分逾通緝時效台北地院判免訴殺人罪部分則因家屬聲請死亡宣告獲准台北地檢署認定林已死亡今處分不起訴檢方調查林輝宏與

北市畜產公司傳弊政風處前總經理涉圖利背信應送辦產發局長林崇傑表示今年4月才知道普盛履約爭議隨即把總經理楊嘉仁換掉資料照2019-06-1015:29:03記者郭安家台北報導台北市政府投資48股權的台北市畜產公司委託廠商每天處理400公噸家禽羽毛卻傳弊案2017年間時任公司業務經理楊嘉仁任內明知得標廠商普盛不具化製資格依舊得標還以150萬元本票150萬元不動產支應300萬元履約保證金涉背信圖利罪楊之後高升為總經理而市府2年才發現問題楊被拔官後再轉任企劃經理產發局長林崇傑回應今年4月才知道普盛履約爭議隨即把總經理換掉楊嘉仁解釋要解約不困難但每天400噸的羽毛量沒有其他業者有能力處理該業者委託其他廠商處理沒有額外收費未造成公司損失才讓普盛繼續做直到去年底和法律顧問討論雙方合議在找到新廠商後解約請繼續往下閱讀議會民政委員會針對畜產公司辦理家禽批發市場事業廢棄物清運履約爭議案專案報告指處理家禽羽毛廠商得標第一天畜產公司就知不具化製資格還可得標只能另外找其他廠商處理下腳料及病死禽而廠商直到得標後1年半多才取得化製證書此外畜產公司明知該廠商積欠台中市政府1千多萬元竟還退回履約擔保金北市府政風處表示前總經理楊嘉仁顯有不當干預為公司處理事務沒有站在公司立場已違背公司利益更圖得第三人利益顯有刑法上的背信嫌疑市場處應該要站在監督立場進一步追究責任移送法辦議員王浩質疑業者2017年得標後畜產公司就知道不符防疫局資格卻又讓業者繼續經營為何沒有糾舉依法處置且合約明訂罰責一毛都沒開罰產發局和市場處都在吃飯睡覺就算職務被拔掉一樣要移送議員林國成批市府未釐清懲處和善後理賠也沒釐清呼籲產發局市場處要有擔當該移送就移送議員苗博雅徐立信認為除了刑事應該還有民事責任沒有扣款普盛長達1年未完整履約提前終止契約卻能完整拿到保證金且當初履約保證金300萬其中150萬採支票支付與契約不符即使帳面沒有損失但重新找人接手也算失損失要求向普盛追討違約金不用抽不用搶現在用APP看新聞保證天天中獎
ans is:  楊嘉仁
tensor(510, device='cuda:0') tensor(-1, device='cuda:0')
model output ans 2 
tensor(510, device='cuda:0') tensor(510, device='cuda:0')
model output ans 2

In [96]:
# method 2 find both top k
BATCH_SIZE = 1
testset = PosTestDataset(test_all_768_x , test_start_pos_labels , test_end_pos_labels , test_binary_y, test_x)
testloader = DataLoader(testset, batch_size=BATCH_SIZE)
pos_model = pos_model.to(device)

pos_model.eval()



with torch.no_grad():
    for data in testloader:

        x, start_pos_labels, end_pos_labels, binary_y = [t.to(device) for t in data[:4]]
        text = data[4]
        mini_batch = x.size()[0]
        
    
        start_pred , end_pred = pos_model(x,x)
        
        start_pred = start_pred.reshape((mini_batch,512))
        end_pred = end_pred.reshape((mini_batch,512))
        
        
        start_pred = torch.nn.functional.softmax(start_pred,dim=1)
        end_pred = torch.nn.functional.softmax(end_pred,dim=1)
        
        

        
        topk = 3
        
        start_pos_labels = torch.argmax(start_pos_labels,dim=1)
        end_pos_labels = torch.argmax(end_pos_labels,dim=1)
        

        
           
        if(binary_y[0] > 0):
            print('===========================')
            print(text[0])
            print('ans is: ', text[0][start_pos_labels-1:end_pos_labels-1])
            print('===========================')
        else:
            continue
        
        start_topk_indices = torch.topk(start_pred, topk).indices
        end_topk_indices = torch.topk(end_pred,topk).indices
        
        for i in range(topk):
            start_index = start_topk_indices[0][i]
            end_index = end_topk_indices[0][i]
            
            start_index -= 1
            end_index -= 1
            
            print('start_index: ' ,start_index)
            print('end_index: ', end_index)
#             if(end_index > start_index or abs(end_index-start_index)>20):
#                 continue
            try:
        
                if(start_index < end_index):
                    print('model output ans ' + str(i) + ' ' +  text[0][start_index:end_index])
                else:
                    print('model output ans ' + str(i) + ' ' +  text[0][end_index:start_index])
            except:
                print('out of range')
#             break
            
#         break

        
        
#         for i in range(len(start_pred)):
#             s_p = start_pred[i]
#             s_a = start_pos_labels[i]
#             s_a = torch.argmax(s_a)
#             s_p = (s_p > thresh_hold).nonzero()
#             s_p = s_p.reshape(s_p.size()[0])
#             if(s_a in s_p):
#                 start_correct += 1
                
#         for i in range(len(end_pred)):
#             e_p = end_pred[i]
#             e_a = end_pos_labels[i]
#             e_a = torch.argmax(e_a)
#             e_p = (e_p > thresh_hold).nonzero()
#             e_p = e_p.reshape(e_p.size()[0])
            
#             if(e_a in e_p):
#                 end_correct += 1
#         break


            
        
        
#         print((start_pred > 0.05).nonzero())
#         print((start_pos_labels > 0).sum(dim=1))
#         print('--------------------')  
#         start_pred = torch.argmax(start_pred,dim=1)
#         end_pred = torch.argmax(end_pred,dim=1)
        
        
        
#         start_pos_labels = torch.argmax(start_pos_labels,dim=1)
#         end_pos_labels = torch.argmax(end_pos_labels,dim=1)
        

        
        
#         end_correct += (end_pred == end_pos_labels).sum().item()
#         end_1 += (end_pred >0).sum().item()
        
#         start_correct += (start_pred == start_pos_labels).sum().item()
#         start_1 += (start_pred>0).sum().item()
        
        
        
        

# print(start_correct , start_1)
# print(end_correct , end_1)


蘋果西打母公司掏空案總經理聲押禁見董座15萬交保大飲總經理孫幼英昨深夜移送新北檢複訊訊後遭聲押禁見記者陳慰慈攝2019-04-1206:00:12記者陳慰慈新北報導生產知名飲料蘋果西打的大西洋飲料公司1213因財報難產爆出掏空疑雲目前股票已暫停交易檢調昨依違反證券交易法非常規交易特別背信罪等罪約談大飲董事長江國貴登記負責人總經理孫幼英實際負責人獨立董事林偉強蘇芸樂財務經理鍾素娥張玉鳳等6人到案蘇芸樂昨晚訊後以3萬5千元交保今凌晨2時許依涉犯重罪有串證逃亡之虞將孫幼英聲押禁見江國貴15萬元林偉強10萬元鍾素娥40萬元張玉鳳25萬元交保大飲公司近日深陷財務經營風暴去年第3季與關係人簽約以64億元取得位於台南佳里高雄湖內及新北市新店等3筆不動產而與大飲交易的國信旭順公司董事名單高度重疊大飲總經理孫幼英身兼國信董事且國信又是大飲旭順的監察人旭順的監察人則是國信公司治理惹議請繼續往下閱讀此外國信旭順公司登記地址也是大飲位於新北市三重區的辦公室等於都是大飲能實質掌控的公司讓外界有左手賣右手的掏空質疑而林偉強蘇芸樂分別擔任國信食品代協理國信食品課長卻又是大飲獨立董事不符金管會規定讓會計師對大飲去年財報給出無法表達意見的結論導致財報必須重編股票也暫停交易檢調懷疑大飲透過土地交易搬錢解救虧損的國信旭順為釐清大飲遭質疑的3筆土地交易是以高買低或是買空賣空的假交易及金錢流向新北地檢署企業犯罪組主任檢察官鄧媛檢察官陳柏文昨指揮調查局新北市調處搜索大飲位於新北市三重區台北市中山區辦公室孫幼英住處等8地查扣相關帳冊等證物全案擴大偵辦中等證物全案擴大偵辦中不用抽不用搶現在用APP看新聞保證天天中獎
ans is:  林偉強
start_index:  tensor(28, device='cuda:0')
end_index:  tensor(31, device='cuda:0')
model output ans 0 理孫幼
start_index:  tensor(510, device='cuda:0')
end_index:  tensor(510, device='cuda:0')
model output ans 1 
start_index:  tensor(-1, device='cuda:0')
end_index:  tensor(-1, device='cuda:0')


獨家竹市2大市場攤商遭詐上億檢起訴攤販協會理事長新竹市竹蓮市場記者蔡彰盛攝2019-03-2523:30:46記者蔡彰盛新竹報導新竹市中正市場得標廠商力揚展業負責人竹市攤販協會理事長林良琪因中正市場經營不順欠下鉅款遂對外聲稱可取得竹蓮市場攤位開始大肆吸金新竹地檢署查出林良琪詐得上億元全案依詐欺罪嫌提起公訴喧騰一時的竹市兩大市場中正市場竹蓮市場多名攤商遭詐騙案經檢警調查發現林良琪2017年6月前負責實際營運竹蓮市場攤販管理戴吟曲則為力揚公司行政經理兼出納負責收受款項請繼續往下閱讀林良琪於2014年間起向竹蓮市場多名攤商謊稱他將在2017年2月間由兒子擔任登記負責人的大舜國際公司參加竹蓮市場招標且他也是投標對手富寶達公司副總有辦法處理竹蓮市場的攤位承租及分配事宜多名攤商陸續繳款但是2017年4月綜合評審時大舜僅獲得第2名事後市府核定由最優申請人富寶達負責營運竹蓮市場林良琪及戴吟曲明知此事卻仍對外佯稱得標者是林之前的合夥人林也有股份眾人買的位置仍在他的股份內屆時將會如期交付攤位致被害人持續交付款項給2人直到2017年10月起林良琪避不見面眾人發覺並無攤位可使用方知受騙而報警2017年2月林良琪另向黃男佯稱想要投標竹蓮市場但申請投標需繳納500萬元保證金黃男掏出現金500萬元給林良琪林開支票作為擔保卻跳票黃男方知受騙以上兩部分案情2人犯罪所得共計2805萬元均依詐欺罪嫌起訴力揚展業公司與市府簽訂BOT案為中正市場得標廠商然因經營不順自2016年6月初即開始跳票在外欠下鉅款的林良琪邀郭女投資中正市場另以800萬元代價出售竹蓮市場1處攤位使用權給郭女郭女為此掏出1500萬元投資款及716萬元攤位使用費林良琪又以需資金製作標案企畫書及打通關節疏通新竹市政府官員等語騙郭女持續出資5千萬元此部分林良琪共騙郭女7216萬元再被檢方依詐欺罪嫌追加起訴許多攤商已經撤出中正市場記者蔡彰盛攝不用抽不用搶現在用APP看新聞保證天天中獎
ans is:  林良琪
start_index:  tensor(83, device='cuda:0')
end_index:  tensor(86, device='cuda:0')
model output ans 0 市攤販
start_index:  tensor(-1, device='cuda:0')
end_index:  tensor(-1, 

成立60年的南港輪胎公司是台灣歷史悠久的輪胎製造商但2014年爆發老臣陳啟清長達20年收取廠商回扣53億元法院依背信罪判陳5年徒刑確定併科罰金1500萬元民事部分雙方和解陳承諾返還516億元不法所得仍被追討上億元利息一審2017年判陳和妻子張月蕉應賠4551萬餘元案件上訴高等法院今天判陳張兩人除4551萬元外應再賠償1395萬元賠償金額總計5946萬餘元仍可上訴退休警官江慶興2014年轉任南港輪胎董事長兩個多月後發現時任資材部經理陳啟清疑向廠商收取回扣向曾任職多年的刑事局報案揪出內鬼陳啟清1978年進入南港輪胎資材部任職從課長一路幹起擔任南港輪胎協理兼任資材部經理時藉機向供應商報關公司討回扣從1995年1月10日至2014年7月9日止陳共計收取回扣53億餘元陳為了隱匿犯罪所得收取回扣期間由妻子張月蕉和不知情親友提供數個帳戶存放回扣款張月蕉待帳戶累積相當金額後再領出現金抱回新竹市住處存放刑事部分一審認為陳貪得無厭背信忘義審酌他認罪依背信罪判陳5年徒刑併科罰金1500萬元張月蕉依洗錢罪判刑1年2月徒刑緩刑3年須給付國庫300萬元陳啟清上訴高院後突然撤回上訴全案確定因陳戶籍地在新竹北檢2016年4月囑託新竹地檢署發監執行民事求償部分陳啟清雖同意吐回5億餘元回扣款但南港輪胎認為扣除本金陳啟清夫婦仍應連帶賠償不法所得利息一審2017年判賠4551萬餘元而南港輪胎就一審判決敗訴部分提起上訴陳張二人也不服判決上訴高院二審今天宣判判陳張應賠償5946萬餘元仍可上訴
ans is:  張月蕉
start_index:  tensor(31, device='cuda:0')
end_index:  tensor(34, device='cuda:0')
model output ans 0 爆發老
start_index:  tensor(106, device='cuda:0')
end_index:  tensor(510, device='cuda:0')
model output ans 1 利息一審2017年判陳和妻子張月蕉應賠4551萬餘元案件上訴高等法院今天判陳張兩人除4551萬元外應再賠償1395萬元賠償金額總計5946萬餘元仍可上訴退休警官江慶興2014年轉任南港輪胎董事長兩個多月後發現時任資材部經理陳啟清疑向廠商收取回扣向曾任職多年的刑事局報案揪出內鬼陳啟清197

記者張文川台北報導屏東縣地下油行負責人郭烈成改名郭盈志僱請施閔毓載送回收豬油等劣質油品賣給強冠公司葉文祥製造黑心油更一審判郭19年4月施1年8月最高法院今天駁回司機施男的上訴判刑1年8月定讞但郭的部分最高院認為更一審認定郭男只是幫助犯恐有違誤撤銷發回高雄高分院更二審至於強冠劣質豬油案強冠董事長葉文祥副總經理戴啟川最高院前年已認定二審判決判葉戴各22年18年徒刑確定其中5年4年可易科罰金強冠公司併科罰金1億2000萬元並沒收犯罪所得但將郭盈志施閔毓的部分發回更一審郭盈志施閔毓被控9次販賣交付含有其他動物成分的劣質油品幫助葉文祥等人製造黑心油更一審判郭19年4月施1年8月最高法院今駁回施的上訴判刑定讞郭則是再次撤銷發回最高法院指出施閔毓雖有上訴但未附上訴理由直接駁回郭盈志方面更一審認定他只是強冠的幫助犯最高法院認為郭在供人食用的原料中攙偽假冒主觀上仍有犯意應論以食安法攙偽假冒罪的正犯更一審未詳加審查認定有所違誤因此發回更審由於正犯的刑度比幫助犯還重意即郭的最終刑期可能會更重不用抽不用搶現在用APP看新聞保證天天中獎點我下載APP按我看活動辦法
ans is:  葉文祥
start_index:  tensor(19, device='cuda:0')
end_index:  tensor(22, device='cuda:0')
model output ans 0 郭烈成
start_index:  tensor(24, device='cuda:0')
end_index:  tensor(233, device='cuda:0')
model output ans 1 郭盈志僱請施閔毓載送回收豬油等劣質油品賣給強冠公司葉文祥製造黑心油更一審判郭19年4月施1年8月最高法院今天駁回司機施男的上訴判刑1年8月定讞但郭的部分最高院認為更一審認定郭男只是幫助犯恐有違誤撤銷發回高雄高分院更二審至於強冠劣質豬油案強冠董事長葉文祥副總經理戴啟川最高院前年已認定二審判決判葉戴各22年18年徒刑確定其中5年4年可易科罰金強冠公司併科罰金1億2000萬元並沒收犯罪所得但將郭盈志施閔毓的部分發回更一審
start_index:  tensor(29, device='cuda:0')
end_index:  tensor(27, device='cuda:0')
model outpu

前立委何智輝因新竹科學園區銅鑼基地等案向國華人壽詐貸4億5000萬元貪汙罪部分一審將他判刑19年上訴二審期間他行賄高院法官後逃亡國華人壽向何男等人求償高院判他與國華董事翁德銘等人賠償2億2500元但遭最高法院廢棄發回判決指出何智輝夫婦在1997年取得久俊工商綜合區開發許可後先後以久俊案土地向國華人壽貸款2億元及2億5000萬元國華人壽董事長張貞松翁德銘指示下屬以不實鑑價及分批貸款方式違法核貸放款造成國華人壽損失國華提告求償台北地院判決何智輝王素筠夫婦和前國華人壽董事長張貞松須連帶賠償國華人壽2億元此部分何智輝夫婦未上訴已告確定前國華董事翁德銘和張貞松須連帶賠償2億5000萬元翁德銘及國華人壽提上訴高院認為何智輝等人共同謀議以國華人壽不法申請貸款致國華受有放貸款項全部無法收回的損害判決何男須與翁德銘等人再賠償2億2500元其中1億6250萬元部分與妻子王素筠連帶賠償但遭最高法院廢棄中時)
ans is:  翁德銘
start_index:  tensor(3, device='cuda:0')
end_index:  tensor(6, device='cuda:0')
model output ans 0 何智輝
start_index:  tensor(4, device='cuda:0')
end_index:  tensor(265, device='cuda:0')
model output ans 1 智輝因新竹科學園區銅鑼基地等案向國華人壽詐貸4億5000萬元貪汙罪部分一審將他判刑19年上訴二審期間他行賄高院法官後逃亡國華人壽向何男等人求償高院判他與國華董事翁德銘等人賠償2億2500元但遭最高法院廢棄發回判決指出何智輝夫婦在1997年取得久俊工商綜合區開發許可後先後以久俊案土地向國華人壽貸款2億元及2億5000萬元國華人壽董事長張貞松翁德銘指示下屬以不實鑑價及分批貸款方式違法核貸放款造成國華人壽損失國華提告求償台北地院判決何智輝王素筠夫婦和前國華人壽董事長張貞松須連帶賠償國華人壽2億元此部分何智輝夫婦未上訴已告確
start_index:  tensor(80, device='cuda:0')
end_index:  tensor(84, device='cuda:0')
model output ans 2 國華董事
▲國安局少校吳宗憲被移送北檢時的

纏訟13年分時度假吸金13億英籍主謀更審判刑6年2006年時以分時度假為幌子的吸金詐騙集團吸金6年共詐得1億2942萬元經過纏訟13年高等法院更一審今依違反銀行法非法經營銀行業務罪對相關人等判刑資料照2019-05-3117:49:58記者張文川台北報導刑事警察局2006年破獲以分時度假為幌子的吸金詐騙集團吸金6年共詐得1億2942萬元經過纏訟13年高等法院更一審今依違反銀行法非法經營銀行業務罪加重改判英國籍主謀倪菲爾6年有期徒刑凌鳳琴等4名經理各判刑3年8月至4年半全案仍可上訴由於案件已審理逾8年適用速審法減刑規定因此倪菲爾等5人皆獲減刑但刑度仍比一審重台灣共犯劉穎谷陳志明各判刑4年6月凌鳳琴李泗源各判3年8月另一經理曾杭皆依常業詐欺罪判刑10月適用96年減刑條例減為5月6人犯罪不法所得一併宣告沒收請繼續往下閱讀判決指出倪菲爾等人於2001年2月起以樂吉美台灣分公司名義對外吸收CVC國際旅遊交換服務會員誘騙會員繳交2萬至70萬元會費宣稱會員可透過交換服務使用全球超過5500家渡假村並享受低廉的旅遊服務實則只是幌子集團還推出5年現金回饋計畫向會員宣稱5年後可以拿回本金或更高的金額實際上卻無法使會員領回會費使會員陷入錯誤而簽約付費集團2006年4月又推出新點子想再剝會員一層皮向會員推銷VIPASIA會員卡佯稱只須再繳交差額就可將先前繳納的CVC會費或其他公司的會籍款項連同這次的會費合併計算總金額後分割成每張價值4萬元的VIPASIA會員卡數張可對外轉售會員卡套利騙會員再次掏錢簽約高院認定樂吉美公司共吸金1億2942萬餘元被害人多達439多人卻未收購其他公司的分時度假權益CVC會員無法取得全球5500家渡假村的使用權也沒有所謂的優惠旅遊且樂吉美收取的會費價格紊亂毫無標準也無法解釋為何客戶先前付給其他公司的會費可以扣抵CVC會費且樂吉美的員工薪資人事費用龐大難以追查資金流向也沒有保證會員可5年取回費用的資金來源因此認定吉美樂違反銀行法詐騙吸金更一審認定倪菲爾等人同時觸犯銀行法與刑法常業詐欺罪從重依銀行法非法經營銀行業務罪論處判刑一審僅依常業詐欺罪輕判4年多未依銀行法論罪因此撤銷改判但因適用速審法給予減刑高院指出此案被害人多達439人不法吸金與詐騙逾億元對社會經濟秩序造成所重大危害公司負責人倪菲爾位居領導決策地位參與程度最深劉穎谷陳志明年薪1115萬陳志明年領逾400萬元2

超跑達人坑富二代勾結竹聯前堂主詐上億超跑達人鄧超鴻左圖涉嫌吸金臉書常曬酷炫跑車照片右圖記者邱俊福翻攝自臉書2019-07-2505:30:00投資黃金失利淪詐騙記者邱俊福台北報導超跑車壇頗具知名度的BetaAuto北塔車業老闆超跑達人鄧超鴻去年間因投資黃金期貨失利資金出現缺口後以誆稱買賣超跑與黃金等投資標的向許多富二代吸金詐騙藉此解決公司財務危機且只要被害人出面索討便由前竹聯幫玄武堂堂主背景的公司股東道克明出面恫嚇刑事局日前將鄧道與公司2員工共4人查緝到案初估吸金詐欺上億元黑幫股東恫嚇被害人警方初步調查38歲鄧男積極擴展超跑市場為打響知名度常舉辦車聚公司全盛時期常停有10多輛超跑每月有數百萬元獲利但去年間鄧男投資黃金期貨損失上億元財務出現危機只好將公司的超跑賣給租賃公司再付租金租回公司同時也向富二代等金主吸金行騙最後公司仍因債務沉重倒閉請繼續往下閱讀鄧男到案後稱要給股東分紅還債才出此下策道男則喊冤表示沒有參與行騙自己也是投資公司的被害人還借錢給鄧男而出面要求被害人不要上門討債只是好心幫忙希望被害人能讓鄧男賺錢後再慢慢歸還沒有恐嚇警方表示鄧男鼓吹被害人以買賣超跑或黃金空運到國外交易賺取價差等投資且為取信被害人將租賃的高價跑車提供被害人作為擔保品惟為能持續詐騙其他被害人便向原被害人假借驗車保養客戶看車等名義將原作為擔保品的高價跑車騙回等事後被害人發覺有異到公司討債道男則出面以黑道背景恫嚇被害人令被害人心生畏懼求償無門目前有9名被害人出面控訴受騙金額高達上億元警方拘提鄧道等4人到案分別在住處及公司查扣現金22萬3000元以及金融卡借據本票等贓證物訊後依組織犯罪詐欺恐嚇等罪嫌移送檢方偵辦其中鄧男被諭令50萬元交保因沒有錢繳付保金遭到羈押男子道克明被送辦記者邱俊福翻攝不用抽不用搶現在用APP看新聞保證天天中獎
ans is:  道克明
start_index:  tensor(22, device='cuda:0')
end_index:  tensor(25, device='cuda:0')
model output ans 0 鄧超鴻
start_index:  tensor(104, device='cuda:0')
end_index:  tensor(107, device='cuda:0')
model output ans 1 uto
start_index:

In [ ]:
BATCH_SIZE = 2
trainset = TrainDataset(train_input_dict,binary_train_y,start_pos_labels_train,end_pos_labels_train)
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)

model = BertForPreTraining.from_pretrained('./chinese_roberta_wwm',config=config)
model = model.to(device)
model.output_hidden_states = True
model.eval()

bi_model = binaryClassification()
bi_model = bi_model.to(device)
bi_model.train()

# start_pos_model = posClassification()
start_pos_model = BiLSTM_Attention()


start_pos_model = start_pos_model.to(device)
start_pos_model = start_pos_model.double()
start_pos_model.train()

# end_pos_model = posClassification()
end_pos_model = BiLSTM_Attention()


end_pos_model = end_pos_model.to(device)
end_pos_model  = end_pos_model.double()
end_pos_model.train()




optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

# binary_loss = nn.BCEWithLogitsLoss()
weights = torch.tensor([0.15,1])
weights = weights.to(device)
weights = weights.float()
binary_loss = nn.CrossEntropyLoss(weight = weights)

# start_pos_loss = nn.KLDivLoss(reduction='mean')
# end_pos_loss = nn.KLDivLoss(reduction='mean')
# pos_weights = np.ones(512)
# pos_weights[0] = 0.3
# pos_weights =  torch.tensor(pos_weights)
# pos_weights = pos_weights.cuda()
# pos_weights = pos_weights.double()
# pos_loss = nn.CrossEntropyLoss(weight = pos_weights)

pos_loss  = nn.KLDivLoss()



EPOCHS = 10  # 幸運數字
for epoch in range(EPOCHS):
    step = 0
    running_loss = 0.0
    for data in trainloader:
        tokens_tensors, segments_tensors, \
        masks_tensors, labels , start_pos_label , end_pos_label = [t.to(device) for t in data]
        bert_outputs = model(input_ids=tokens_tensors, 
                                token_type_ids=segments_tensors, 
                                attention_mask=masks_tensors)
        
        
        
        
        hidden_state = bert_outputs[2]
        bert_all_768 = hidden_state[0]
        cls = bert_all_768[:,0,:]
        
#         binary classification part
        binary_classify_output = bi_model(cls)
        binary_classify_output = torch.nn.functional.softmax(binary_classify_output,dim=1)
        labels = labels.long()
        bi_loss = binary_loss(binary_classify_output,labels)
        
        
#         pos predict
        bert_all_768 = bert_all_768.double()
    
        
#         start_pos_pred_output = start_pos_model(bert_all_768)
#         start_pos_pred_output = torch.nn.functional.softmax(start_pos_pred_output,dim=1)
#         start_loss = pos_loss(start_pos_pred_output,start_pos_label)
        
#         end_pos_pred_output = end_pos_model(bert_all_768)
#         end_pos_pred_output = torch.nn.functional.softmax(end_pos_pred_output,dim=1)
#         end_loss = pos_loss(end_pos_pred_output,end_pos_label)
                
        
        start_pos_pred_output , attention = start_pos_model(bert_all_768)
        start_pos_pred_output = torch.nn.functional.softmax(start_pos_pred_output,dim=1)
        start_loss = pos_loss(start_pos_pred_output,start_pos_label)
        
        end_pos_pred_output , attention = end_pos_model(bert_all_768)
        end_pos_pred_output = torch.nn.functional.softmax(end_pos_pred_output,dim=1)
        end_loss = pos_loss(end_pos_pred_output,end_pos_label)
            
        
#         sum the loss
        start_loss = start_loss.double()
        end_loss = end_loss.double()
        loss = bi_loss + start_loss + end_loss
        
        
#         output, attention = model(x)

        
        
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
    torch.save(start_pos_model, 'start_pos_model.pkl')
    torch.save(end_pos_model, 'end_pos_model.pkl')
    torch.save(bi_model,'binary_model.pkl')
    print('[epoch %d] loss: %.3f' %(epoch + 1, running_loss))

In [ ]:
labels.shape

In [ ]:
test_input_dict = tokenizer.batch_encode_plus(test_x, 
                                         add_special_tokens=True,
                                         max_length=512,
                                         return_special_tokens_masks=True,
                                         pad_to_max_length=True,
                                         return_tensors='pt')

with open('./TB/test_input_dict.pickle', 'wb') as handle:
    pickle.dump(test_input_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# with open('./TB/test_input_dict.pickle', 'rb') as handle:
#     test_input_dict = pickle.load(handle)

In [ ]:
BATCH_SIZE = 4
testset = TrainDataset(test_input_dict,binary_test_y,start_pos_labels_test,end_pos_labels_test)
testloader = DataLoader(testset, batch_size=BATCH_SIZE)


In [ ]:
model.eval()
bi_model.eval()
start_pos_model.eval()
end_pos_model.eval()

correct_count = 0
guess_1_count = 0

start_correct_count = 0
end_correct_count = 0

start_non_zero = 0
end_non_zero = 0

with torch.no_grad():
    for data in testloader:
        tokens_tensors, segments_tensors, \
        masks_tensors, labels , start_pos_label , end_pos_label = [t.to(device) for t in data]
        bert_outputs = model(input_ids=tokens_tensors, 
                                token_type_ids=segments_tensors, 
                                attention_mask=masks_tensors)
        
        hidden_state = bert_outputs[2]
        bert_all_768 = hidden_state[0]
        cls = bert_all_768[:,0,:]
        
        binary_classify_output = bi_model(cls)
        binary_classify_output = torch.argmax(binary_classify_output,dim=1)
        correct_count += (labels == binary_classify_output).sum().item()
        guess_1_count += (binary_classify_output==1).sum().item()
        
        
        
        
#         pos predict
        bert_all_768 = bert_all_768.double()
    
        start_pos_pred_output = start_pos_model(bert_all_768)
        start_pos_pred_output = torch.nn.functional.softmax(start_pos_pred_output, dim = 1)
        
        start_pred = torch.argmax(start_pos_pred_output,dim = 1)
        
        start_non_zero += (start_pred>0).sum().item()
#         start_ans = torch.argmax(start_pos_label,dim = 1)

    
    
        start_correct_count += (start_pred == start_pos_label).sum().item()
        

        end_pos_pred_output = end_pos_model(bert_all_768)
        end_pos_pred_output = torch.nn.functional.softmax(end_pos_pred_output, dim = 1)
        end_pred = torch.argmax(end_pos_pred_output,dim = 1)
#         end_ans = torch.argmax(end_pos_label,dim = 1)
        
#         print(end_pred)
#         print(end_ans)
        
        end_correct_count += (end_pred == end_pos_label).sum().item()
        end_non_zero += (end_pred>0).sum().item()

#         break

#         print(start_pos_label.sum(dim=1))

#         if(start_pos_label.sum()>=1):
#             print(start_pos_pred_output)
#             print((start_pos_label[1] != 0).nonzero())
#             print(torch.argmax(start_pos_pred_output[1]))
#             break
        
        
        
print(correct_count)
print(guess_1_count)
print(start_correct_count,start_non_zero)
print(end_correct_count,end_non_zero)


In [ ]:
def get_predictions(model, dataloader, compute_acc=False):
    predictions = None
    predictions_withoutmax = None
    correct = 0
    total = 0
    with torch.no_grad():
        # 遍巡整個資料集
        for data in dataloader:
            # 將所有 tensors 移到 GPU 上
            if next(model.parameters()).is_cuda:
                data = [t.to("cuda:0") for t in data if t is not None]
            
            tokens_tensors, segments_tensors, masks_tensors = data[:3]
            outputs = model(input_ids=tokens_tensors, 
                            token_type_ids=segments_tensors, 
                            attention_mask=masks_tensors)
            
            logits = outputs[0]
            after_softmax = F.softmax(logits.data, dim=1)
            _, pred = torch.max(after_softmax, 1)
            
            if compute_acc:
                labels = data[3]
                total += labels.size(0)
                correct += (pred == labels).sum().item()
                
            # 將當前 batch 記錄下來
            if predictions is None:
                predictions = pred
            else:
                predictions = torch.cat((predictions, pred))
    
            if predictions_withoutmax is None:
                predictions_withoutmax = after_softmax
            else:
                predictions_withoutmax = torch.cat((predictions_withoutmax,after_softmax))
    
    if compute_acc:
        acc = correct / total
        return predictions, acc
    return predictions_withoutmax

In [ ]:
test_input_dict = tokenizer.batch_encode_plus(test_x, 
                                         add_special_tokens=True,
                                         max_length=512,
                                         return_special_tokens_masks=True,
                                         pad_to_max_length=True,
                                         return_tensors='pt')
BATCH_SIZE = 64
testset = TestDataset(test_input_dict)
testloader = DataLoader(testset, batch_size=BATCH_SIZE)
predictions = get_predictions(model, testloader)

In [ ]:

pred = predictions.cpu().data.numpy()
pred = np.argmax(pred, axis=1)
accuracy = (pred == binary_test_y).mean()
print('Your test accuracy is %.6f' % (accuracy * 100))

In [ ]:
count = 0
for i in range(len(pred)):
    label = binary_test_y[i]
    _pred = pred[i]
    if(label ==1 and _pred != label):
        count += 1

In [ ]:
count / sum(binary_test_y)